<a href="https://colab.research.google.com/github/ykykyk112/machine_learning/blob/main/sklearn_exercise/Ensemble_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

test = np.load('/content/drive/MyDrive/Colab_Notebooks/quickdraw/bird.npy')
print(test.shape)

In [ ]:
idx = np.arange(test.shape[0])
rnd_idx = np.random.choice(idx, 10000, replace=False)
t = test[rnd_idx]
t[0:10000].shape
del idx

In [ ]:
def make_ndarray(target_list, each_size) :
    # 784 means bitmap size
    data = np.empty((len(target_list)*each_size, 784))
    target = np.empty((len(target_list)*each_size))
    for label, filename in enumerate(target_list) :
        npy = np.load('/content/drive/MyDrive/Colab_Notebooks/quickdraw/{}.npy'.format(filename))
        tmp = np.arange(npy.shape[0])
        idx = np.random.choice(tmp, each_size, replace=False)
        data[label*each_size:(label+1)*each_size] = npy[idx]
        target[label*each_size:(label+1)*each_size] = label
        print('{} is Complete'.format(filename))
    s_idx = np.arange(target.shape[0])
    np.random.shuffle(s_idx)
    data = data[s_idx]
    target = target[s_idx]
    print('result shape : {0} & {1}'.format(data.shape, target.shape))
    return data, target

target_list = ['ant', 'bicycle', 'bird', 'butterfly', 'camel', 'clock', 'cow', 'diamond']
data, target = make_ndarray(target_list=target_list, each_size=10000)
print(data.shape, target.shape)

In [ ]:
def show_image(index) :
    plt.imshow(data[index].reshape(28, 28), cmap='binary')
    plt.title(target[index])
show_image(40)

In [ ]:
from sklearn import model_selection
X_train, X_test, y_train, y_test = model_selection.train_test_split(data, target, test_size = 0.2, random_state=42)

In [ ]:
from sklearn import tree
dt_clf = tree.DecisionTreeClassifier(random_state=42)
dt_clf.fit(X_train, y_train)
print(dt_clf.score(X_test, y_test))

In [ ]:
from sklearn import svm
svm_clf = svm.LinearSVC(random_state=42)
svm_clf.fit(X_train, y_train)
print(svm_clf.score(X_test, y_test))

In [ ]:
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier # Neural network

In [ ]:
# classifier generation
random_clf = RandomForestClassifier(random_state=42)
mlp_clf = MLPClassifier(random_state=42)
ext_clf = ExtraTreesClassifier(random_state=42)

In [ ]:
# Training  classifiers
random_clf.fit(X_train, y_train)
mlp_clf.fit(X_train, y_train)
ext_clf.fit(X_train, y_train)

In [ ]:
print(random_clf.score(X_test, y_test))
print(mlp_clf.score(X_test, y_test))
print(ext_clf.score(X_test, y_test))

In [ ]:
from sklearn.ensemble import VotingClassifier
named_estimators = [('rf', random_clf),
                    ('mlp', mlp_clf),
                    ('svc', svm_clf),
                    ('ext', ext_clf),
                    ('dt', dt_clf)]

In [ ]:
voting_clf = VotingClassifier(estimators=named_estimators, voting='hard')
voting_clf.fit(X_train, y_train)

In [ ]:
voting_score = voting_clf.score(X_test, y_test)
voting_score

In [ ]:
voting_clf.set_params(svc = None)
del voting_clf.estimators_[2]

In [ ]:
new_voting_score = voting_clf.score(X_test, y_test)
print("Old VotingClassifier : ", voting_score, "\nNew VotingClassifier : ", new_voting_score)